# Workshop Part 2: The "Brain" (The Agent)

In this notebook, we act as the **Host**. We will connect to our running MCP server and implement the **Evaluator-Optimizer** pattern.

In [ ]:
# Install client dependencies
%pip install openai mcp httpx

In [ ]:
import os
import asyncio
from getpass import getpass
from openai import OpenAI
from mcp import ClientSession, StdioServerParameters
from mcp.client.sse import sse_client
import httpx

# 1. Setup API Key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = getpass("Enter your OpenAI API Key: ")

client = OpenAI(api_key=api_key)
SERVER_URL = "http://localhost:8000/sse"

# 2. Verify Server is Alive before starting
try:
    print(f"Checking connection to {SERVER_URL}...")
    print("✅ Configured. Ensure your terminal running 'fastmcp' is active.")
except Exception:
    print("⚠️ Warning: Could not verify server. Make sure it's running!")

In [ ]:
def clean_code_block(text: str) -> str:
    """Robustly extracts code from markdown fences."""
    text = text.strip()
    if text.startswith("```python"):
        text = text[9:]
    elif text.startswith("```"):
        text = text[3:]
    if text.endswith("```"):
        text = text[:-3]
    return text.strip()

async def agent_loop(user_prompt):
    messages = [
        {"role": "system", "content": "You are a Python coding assistant. Output ONLY raw python code. No markdown formatting."}
    ]
    messages.append({"role": "user", "content": user_prompt})

    print("🔌 Connecting to MCP Server...")
    try:
        # Standard MCP SSE Client Context Manager
        async with sse_client(SERVER_URL) as (read_stream, write_stream):
            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                print("✅ Connected to CodeAssistant!")
                
                for attempt in range(3):
                    print(f"\n--- Attempt {attempt + 1} ---")
                    
                    # 1. GENERATE
                    response = client.chat.completions.create(
                        model="gpt-4o", 
                        messages=messages
                    )
                    raw_content = response.choices[0].message.content
                    generated_code = clean_code_block(raw_content)
                    
                    print(f"Generated Code:\n{generated_code[:100]}... [truncated]")
                    
                    # 2. EVALUATE (Call remote tool)
                    lint_result = await session.call_tool("lint_code", arguments={"code": generated_code})
                    
                    # Inspect result - MCP returns a list of content objects
                    lint_output = lint_result.content[0].text
                    
                    if lint_output == "OK":
                        print("✅ Linter Passed!")
                        
                        # 3. EXECUTE
                        exec_result = await session.call_tool("run_safe_python", arguments={"code": generated_code})
                        print(f"\n🏆 Final Execution Output:\n{exec_result.content[0].text}")
                        return generated_code
                    
                    else:
                        print(f"❌ Linter Failed: {lint_output}")
                        # Feed error back to LLM
                        messages.append({"role": "assistant", "content": generated_code})
                        messages.append({"role": "user", "content": f"Fix this error: {lint_output}"})
                
                print("❌ Failed to optimize code after 3 attempts.")
    except httpx.ConnectError:
        print(f"\n⛔ CONNECTION ERROR: Could not connect to {SERVER_URL}.")
        print("👉 Verify you ran: 'fastmcp run server.py --transport sse --port 8000' in your terminal.")
    except Exception as e:
        print(f"\n⛔ UNEXPECTED ERROR: {e}")

In [ ]:
# Trigger the Agent
task = "Write a script that prints the first 5 Fibonacci numbers but make a syntax error by forgetting a colon."

# Note: In Jupyter, top-level await is supported
await agent_loop(task)